In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix,accuracy_score,r2_score
from sklearn.linear_model import LogisticRegression

In [12]:
data=pd.read_excel("C:/Users/rajmh/Downloads/Logistic_File_2.xlsx")


In [13]:
df=pd.DataFrame(data)

In [14]:
df

,Income,CreditScore,Employment,LoanAmount,Age,MaritalStatus,Default
0,5.2,700,employed,25.0,35,married,0
1,3.0,580,self,40.0,29,single,1
2,4.5,NaN,employed,35.0,40,married,0
3,NaN,600,unemployed,20.0,22,single,1
4,6.0,720,employed,15.0,45,divorced,0
5,2.8,550,self,30.0,NaN,single,1
6,4.2,680,employed,NaN,31,married,0
7,3.6,610,unemployed,28.0,26,married,1


In [15]:
## split column into dependent and independent

In [16]:
# df['Employment']=df['Employment'].str.strip()   
# df['MaritalStatus']=df['MaritalStatus'].str.strip()

for col in ['Employment', 'MaritalStatus']:
    df[col] = df[col].str.strip()

In [17]:
X=df.drop('Default',axis=1)
y=df['Default']

In [18]:
## preprocessing fill num with mean and categorical with mode

In [19]:
num_feature=['Income','CreditScore','LoanAmount','Age']
cat_feature=['Employment','MaritalStatus']


In [20]:
num_transformation=Pipeline(steps=[
    ('impute',SimpleImputer(strategy='mean')),
    ('scaler',StandardScaler())   
])

In [21]:
cat_transformation=Pipeline(steps=[
    ('impute',SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
    
])

In [22]:
## combine both to transform

In [23]:
combine=ColumnTransformer(
    transformers=[
   ('num',num_transformation,num_feature),
   ('cat',cat_transformation,cat_feature) ])

In [33]:
combine

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('impute', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Income', 'CreditScore', 'LoanAmount',
                                  'Age']),
                                ('cat',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Employment', 'MaritalStatus'])])

In [25]:
model = Pipeline(steps=[
    ('preprocessor', combine),
    ('classifier', LogisticRegression())])

In [26]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)

In [27]:
model.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Income', 'CreditScore',
                                                   'LoanAmount', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Employment',
                                                   'MaritalStatus'])])),
                ('classifier', LogisticRegression())])

In [28]:
y_pred = model.predict(X_test)

In [29]:
X_test

,Income,CreditScore,Employment,LoanAmount,Age,MaritalStatus
1,3.0,580,self,40.0,29,single
5,2.8,550,self,30.0,NaN,single
0,5.2,700,employed,25.0,35,married


In [30]:
y_pred

array([1, 1, 0], dtype=int64)

In [31]:
accuracy = accuracy_score(y_test, y_pred)

In [32]:
accuracy

1.0